<br/>

<img src="https://uploads-ssl.webflow.com/614b1fe22fa8b90ef41aeffe/6265cb48f9496b1cefc9ab75_logotipo-mbit-39.png" width="200px" align="right" CLASS="TextWrap" style="background-color:#2a3f3f;">

<h1><font color="#2a3f3f" size=5></font></h1>

<br/>
<div style="text-align: right">
<font color="#2a3f3f" size=3>Grupo 1 - MSF - Predicción de aumento de cuotas de socios </font><br>
</div>

En este proyecto vamos a analizar los datos aportados por Medicos Sin Fronteras y preparar un modelo para la predicción de aumento de cuotas de socios.

### Fase 0. Configurar entorno local

Primero configuraremos nuestro entorno local

In [2]:
%stop_session

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
There is no current session.


In [4]:
%additional_python_modules "seaborn"

Additional python modules to be included:
seaborn


In [1]:
import pandas as pd
import numpy as np
# from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as f
from pyspark.sql.functions import col,isnan, when, count
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, Word2Vec, HashingTF, IDF, Normalizer

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::048549020029:role/LabRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: a2d57e27-820d-4c93-89fe-ab7d957ed12d
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
--additional-python-modules seaborn
Waiting for session a2d57e27-820d-4c93-89fe-ab7d957ed12d to get into ready status...
Session a2d57e27-820d-4c93-89fe-ab7d957ed12d has been created.



In [2]:
# Establecemos que se muestren el nº total de filas y columnas en las salidas
pd.set_option("max_columns", None)
pd.set_option("max_rows", None)

## Fase 1: Carga de datos desde S3

#### Carga los datos de la tabla "Contactos"

In [3]:
df_contactos = pd.read_parquet("s3://marta-tmf-oct22-msf-data/MSF_Contact.parquet/")

In [4]:
df_contactos.head(n=1)

                   id  msf_seniority__c npo02__best_gift_year__c  \
0  0033Y00002uqxIJQAY              19.0                     2005   

  msf_birthyear__c msf_entrycampaign__c  \
0                    7013Y000001mrCzQAI   

  msf_firstcampaignrecurringdonorchannel__c leadsource  \
0                                                 Otro   

  msf_firstcampaigncolaborationchannel__c  msf_returnedmail__c  \
0                                    Otro                False   

   npo02__averageamount__c msf_isactivedonor__c msf_isactiverecurringdonor__c  \
0                      0.0            Exdonante                         Nunca   

   npsp__deceased__c msf_begindatemsf__c msf_fechacambiolevelrelacion__c  \
0              False          2005-02-28                      2020-03-28   

  msf_datefirstdonation__c msf_datefirstrecurringdonorquota__c  \
0               2005-01-07                                None   

  msf_datelastrecurringdonorquota__c msf_datelastdonation__c  \
0            

In [5]:
df_contactos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803419 entries, 0 to 1803418
Columns: 139 entries, id to msf_commitment__c
dtypes: bool(57), float64(36), object(46)
memory usage: 1.2+ GB


In [6]:
columnas_contactos =  df_contactos.columns.tolist()
columnas_contactos

['id', 'msf_seniority__c', 'npo02__best_gift_year__c', 'msf_birthyear__c', 'msf_entrycampaign__c', 'msf_firstcampaignrecurringdonorchannel__c', 'leadsource', 'msf_firstcampaigncolaborationchannel__c', 'msf_returnedmail__c', 'npo02__averageamount__c', 'msf_isactivedonor__c', 'msf_isactiverecurringdonor__c', 'npsp__deceased__c', 'msf_begindatemsf__c', 'msf_fechacambiolevelrelacion__c', 'msf_datefirstdonation__c', 'msf_datefirstrecurringdonorquota__c', 'msf_datelastrecurringdonorquota__c', 'msf_datelastdonation__c', 'npsp__largest_soft_credit_date__c', 'npsp__first_soft_credit_date__c', 'msf_entrydatecurrentrecurringdonor__c', 'npsp__last_soft_credit_date__c', 'msf_firstentrydaterecurringdonor__c', 'npo02__firstclosedate__c', 'msf_lastrecurringdonationdate__c', 'npo02__lastclosedate__c', 'msf_lastdonationunique__c', 'gender__c', 'msf_crmexternalid__c', 'msf_languagepreferer__c', 'npo02__largestamount__c', 'npo02__smallestamount__c', 'npsp__first_soft_credit_amount__c', 'npo02__lastoppamou

In [7]:
# Se revisa el nº total de registros y columnas de la tabla "Contactos"
df_contactos.shape

(1803419, 139)


In [8]:
# Se analizan la cantidad de nulos en cada variable del dataset
nulos = df_contactos.isnull().sum()
nulos

id                                                  0
msf_seniority__c                                    0
npo02__best_gift_year__c                            0
msf_birthyear__c                                    0
msf_entrycampaign__c                                0
msf_firstcampaignrecurringdonorchannel__c           0
leadsource                                          0
msf_firstcampaigncolaborationchannel__c             0
msf_returnedmail__c                                 0
npo02__averageamount__c                             0
msf_isactivedonor__c                                0
msf_isactiverecurringdonor__c                       0
npsp__deceased__c                                   0
msf_begindatemsf__c                                 1
msf_fechacambiolevelrelacion__c                  2204
msf_datefirstdonation__c                      1195087
msf_datefirstrecurringdonorquota__c            858069
msf_datelastrecurringdonorquota__c             858069
msf_datelastdonation__c     

In [10]:
# Comprobamos si existen duplicados
print(len(df_contactos))
print(len(df_contactos.drop_duplicates()))

1803419
1803419


<div class="alert alert-block alert-success">
No existen duplicados en el dataset
</div>

#### Carga los datos de la tabla "recurring_donation"

In [5]:
df_re_donation = pd.read_parquet("s3://marta-tmf-oct22-msf-data/MSF_RecurringDonation.parquet/")

In [12]:
df_re_donation.head(n=1)

                   id  isdeleted  msf_annualizedquota__c  \
0  a093Y00001RhiLNQAZ      False                    72.0   

  msf_cancelationdate__c msf_cancelationreason__c msf_currentcampaign__c  \
0             2011-02-04      Impago aportaciones     7013Y000001mqt9QAA   

  msf_currentleadsource1__c msf_currentquotamodification__c  \
0             Telemarketing              a1y3Y000004sW6pQAE   

  msf_leadsource1__c msf_memberid__c  npe03__amount__c   npe03__contact__c  \
0              Cupón         9969053               6.0  0033Y00002uppXLQAY   

  npe03__date_established__c npe03__installment_period__c  \
0                 2011-01-07                      Monthly   

  npe03__last_payment_date__c npe03__next_payment_date__c  \
0                  2011-03-01                        None   

  npe03__open_ended_status__c  npe03__paid_amount__c  \
0                      Closed                    0.0   

  npe03__recurring_donation_campaign__c  npe03__total_paid_installments__c  \
0    

In [13]:
df_re_donation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198207 entries, 0 to 1198206
Data columns (total 21 columns):
 #   Column                                 Non-Null Count    Dtype  
---  ------                                 --------------    -----  
 0   id                                     1198207 non-null  object 
 1   isdeleted                              1198207 non-null  bool   
 2   msf_annualizedquota__c                 1198207 non-null  float64
 3   msf_cancelationdate__c                 715201 non-null   object 
 4   msf_cancelationreason__c               1198207 non-null  object 
 5   msf_currentcampaign__c                 1198207 non-null  object 
 6   msf_currentleadsource1__c              1198207 non-null  object 
 7   msf_currentquotamodification__c        1198207 non-null  object 
 8   msf_leadsource1__c                     1198207 non-null  object 
 9   msf_memberid__c                        1198207 non-null  object 
 10  npe03__amount__c                       119

In [14]:
columnas_re_donation =  df_re_donation.columns.tolist()
columnas_re_donation

['id', 'isdeleted', 'msf_annualizedquota__c', 'msf_cancelationdate__c', 'msf_cancelationreason__c', 'msf_currentcampaign__c', 'msf_currentleadsource1__c', 'msf_currentquotamodification__c', 'msf_leadsource1__c', 'msf_memberid__c', 'npe03__amount__c', 'npe03__contact__c', 'npe03__date_established__c', 'npe03__installment_period__c', 'npe03__last_payment_date__c', 'npe03__next_payment_date__c', 'npe03__open_ended_status__c', 'npe03__paid_amount__c', 'npe03__recurring_donation_campaign__c', 'npe03__total_paid_installments__c', 'npsp4hub__payment_method__c']


In [15]:
# Se revisa el nº total de registros y columnas de la tabla "donaciones recurrentes"
df_re_donation.shape

(1198207, 21)


In [16]:
# Se analizan la cantidad de nulos en cada variable del dataset
nulos = df_re_donation.isnull().sum()
nulos

id                                            0
isdeleted                                     0
msf_annualizedquota__c                        0
msf_cancelationdate__c                   483006
msf_cancelationreason__c                      0
msf_currentcampaign__c                        0
msf_currentleadsource1__c                     0
msf_currentquotamodification__c               0
msf_leadsource1__c                            0
msf_memberid__c                               0
npe03__amount__c                              0
npe03__contact__c                             0
npe03__date_established__c                    0
npe03__installment_period__c                  0
npe03__last_payment_date__c               75610
npe03__next_payment_date__c              714905
npe03__open_ended_status__c                   0
npe03__paid_amount__c                      1899
npe03__recurring_donation_campaign__c         0
npe03__total_paid_installments__c          1899
npsp4hub__payment_method__c             

In [17]:
# Comprobamos si existen duplicados
print(len(df_re_donation))
print(len(df_re_donation.drop_duplicates()))

1198207
1198207


<div class="alert alert-block alert-success">
No existen duplicados en el dataset
</div>

#### Carga los datos de la tabla "Modificacion de cuota"

In [6]:
df_mod_cuota = pd.read_parquet("s3://marta-tmf-oct22-msf-data/MSF_QuotaModification.parquet/")

In [26]:
df_mod_cuota.head(n=1)

                   id  isdeleted                name msf_recurringdonation__c  \
0  a1y3Y000001sAtxQAE      False  A - 15948014621775       a093Y00001RZ7cgQAD   

  msf_campaigninfluence__c  msf_changeamount__c  msf_changeannualizedquota__c  \
0       7013Y000001mquNQAQ                  5.0                          60.0   

  msf_changetype__c msf_leadsource1__c msf_leadsource2__c msf_leadsource3__c  \
0          Increase       Teléfono web       Teléfono web           Teléfono   

   msf_newamount__c  msf_newannualizedquota__c msf_newrecurringperiod__c  \
0              20.0                      240.0                   Monthly   

     msf_contactid__c msf_changedate__c  
0  0033Y00002uNQ6CQAW        2020-04-02


In [27]:
df_mod_cuota.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003019 entries, 0 to 2003018
Data columns (total 16 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id                            object 
 1   isdeleted                     bool   
 2   name                          object 
 3   msf_recurringdonation__c      object 
 4   msf_campaigninfluence__c      object 
 5   msf_changeamount__c           float64
 6   msf_changeannualizedquota__c  float64
 7   msf_changetype__c             object 
 8   msf_leadsource1__c            object 
 9   msf_leadsource2__c            object 
 10  msf_leadsource3__c            object 
 11  msf_newamount__c              float64
 12  msf_newannualizedquota__c     float64
 13  msf_newrecurringperiod__c     object 
 14  msf_contactid__c              object 
 15  msf_changedate__c             object 
dtypes: bool(1), float64(4), object(11)
memory usage: 231.1+ MB


In [28]:
columnas_mod_cuota =  df_mod_cuota.columns.tolist()
columnas_mod_cuota

['id', 'isdeleted', 'name', 'msf_recurringdonation__c', 'msf_campaigninfluence__c', 'msf_changeamount__c', 'msf_changeannualizedquota__c', 'msf_changetype__c', 'msf_leadsource1__c', 'msf_leadsource2__c', 'msf_leadsource3__c', 'msf_newamount__c', 'msf_newannualizedquota__c', 'msf_newrecurringperiod__c', 'msf_contactid__c', 'msf_changedate__c']


In [29]:
# Se revisa el nº total de registros y columnas de la tabla "donaciones recurrentes"
df_mod_cuota.shape

(2003019, 16)


In [30]:
# Se analizan la cantidad de nulos en cada variable del dataset
nulos = df_mod_cuota.isnull().sum()
nulos

id                                0
isdeleted                         0
name                              0
msf_recurringdonation__c          0
msf_campaigninfluence__c          0
msf_changeamount__c               0
msf_changeannualizedquota__c      0
msf_changetype__c                 0
msf_leadsource1__c                0
msf_leadsource2__c                0
msf_leadsource3__c                0
msf_newamount__c                  0
msf_newannualizedquota__c         0
msf_newrecurringperiod__c         0
msf_contactid__c                  0
msf_changedate__c               186
dtype: int64


In [31]:
# Comprobamos si existen duplicados
print(len(df_mod_cuota))
print(len(df_mod_cuota.drop_duplicates()))

2003019
2003019


<div class="alert alert-block alert-success">
No existen duplicados en el dataset
</div>

#### Carga los datos de la tabla "Campaña"

In [32]:
df_Campaign = pd.read_parquet("s3://marta-tmf-oct22-msf-data/MSF_Campaign.parquet/")

In [33]:
df_Campaign.head(n=1)

                   id msf_attribute_1__c msf_attribute_2__c  \
0  7013Y000001mrHWQAY   Afiliación Leads                      

  msf_attribute_3__c msf_attribute_4__c msf_attribute_5__c  \
0                                                            

  msf_campaigndonationreporting__c msf_campaignentryreporting__c  \
0              23-Digital Orgánico           23-Digital Orgánico   

  msf_canalsalidaconcatenado__c  msf_isemergency__c msf_isonline__c  \
0                  Afiliación -               False              Si   

                 msf_objective__c msf_objectivepublic__c  \
0  Captación de socios o donantes                          

  msf_outboundchannel1__c msf_outboundchannel2__c msf_ownby__c  \
0              Afiliación                              Digital   

  msf_previousstepchannel__c msf_promoterindividual__c msf_provider__c  \
0                                                                        

    msf_segment__c msf_thematic__c             ownerid        rec

In [34]:
df_Campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11501 entries, 0 to 11500
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   id                                11501 non-null  object
 1   msf_attribute_1__c                11501 non-null  object
 2   msf_attribute_2__c                11501 non-null  object
 3   msf_attribute_3__c                11501 non-null  object
 4   msf_attribute_4__c                11501 non-null  object
 5   msf_attribute_5__c                11501 non-null  object
 6   msf_campaigndonationreporting__c  11501 non-null  object
 7   msf_campaignentryreporting__c     11501 non-null  object
 8   msf_canalsalidaconcatenado__c     11501 non-null  object
 9   msf_isemergency__c                11501 non-null  bool  
 10  msf_isonline__c                   11501 non-null  object
 11  msf_objective__c                  11501 non-null  object
 12  msf_objectivepubli

In [35]:
columnas_Campaign =  df_Campaign.columns.tolist()
columnas_Campaign

['id', 'msf_attribute_1__c', 'msf_attribute_2__c', 'msf_attribute_3__c', 'msf_attribute_4__c', 'msf_attribute_5__c', 'msf_campaigndonationreporting__c', 'msf_campaignentryreporting__c', 'msf_canalsalidaconcatenado__c', 'msf_isemergency__c', 'msf_isonline__c', 'msf_objective__c', 'msf_objectivepublic__c', 'msf_outboundchannel1__c', 'msf_outboundchannel2__c', 'msf_ownby__c', 'msf_previousstepchannel__c', 'msf_promoterindividual__c', 'msf_provider__c', 'msf_segment__c', 'msf_thematic__c', 'ownerid', 'recordtypeid', 'status']


In [36]:
# Se revisa el nº total de registros y columnas de la tabla "donaciones recurrentes"
df_Campaign.shape

(11501, 24)


In [37]:
# Se analizan la cantidad de nulos en cada variable del dataset
nulos = df_Campaign.isnull().sum()
nulos

id                                  0
msf_attribute_1__c                  0
msf_attribute_2__c                  0
msf_attribute_3__c                  0
msf_attribute_4__c                  0
msf_attribute_5__c                  0
msf_campaigndonationreporting__c    0
msf_campaignentryreporting__c       0
msf_canalsalidaconcatenado__c       0
msf_isemergency__c                  0
msf_isonline__c                     0
msf_objective__c                    0
msf_objectivepublic__c              0
msf_outboundchannel1__c             0
msf_outboundchannel2__c             0
msf_ownby__c                        0
msf_previousstepchannel__c          0
msf_promoterindividual__c           0
msf_provider__c                     0
msf_segment__c                      0
msf_thematic__c                     0
ownerid                             0
recordtypeid                        0
status                              0
dtype: int64


In [38]:
# Comprobamos si existen duplicados
print(len(df_Campaign))
print(len(df_Campaign.drop_duplicates()))

11501
11501


<div class="alert alert-block alert-success">
No existen duplicados en el dataset
</div>

#### Carga los datos de la tabla "Oportunidad"

In [39]:
df_oportunidad = pd.read_parquet("s3://marta-tmf-oct22-msf-data/MSF_Opportunity.parquet/")

Error: Interpreter died:


ERROR:fake_shell:execute_reply
Traceback (most recent call last):
  File "/tmp/1473154913106944811", line 106, in execute
    exec(code, global_dict)
  File "<stdin>", line 1, in <module>
NameError: name 'df_contactosdf_contactos' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/1473154913106944811", line 200, in execute_request
    result = node.execute()
  File "/tmp/1473154913106944811", line 115, in execute
    raise ExecutionError(sys.exc_info())
ExecutionError: (<class 'NameError'>, NameError("name 'df_contactosdf_contactos' is not defined"), <traceback object at 0x7fb00aefb410>)


In [ ]:
df_oportunidad.head(n=1)

In [ ]:
df_oportunidad.info()

In [ ]:
columnas_oportunidad =  df_oportunidad.columns.tolist()
columnas_oportunidad

In [ ]:
# Se revisa el nº total de registros y columnas de la tabla "oportunidad"
df_oportunidad.shape

In [ ]:
# Se analizan la cantidad de nulos en cada variable del dataset
nulos = df_oportunidad.isnull().sum()
nulos

In [ ]:
# Comprobamos si existen duplicados
print(len(df_oportunidad))
print(len(df_oportunidad.drop_duplicates()))

<div class="alert alert-block alert-success">
No existen duplicados en el dataset
</div>

#### Carga los datos de la tabla "Tareas"

In [40]:
df_tareas = pd.read_parquet("s3://marta-tmf-oct22-msf-data/MSF_Task.parquet/")

Error: Interpreter died:



In [ ]:
df_tareas.head(n=1)

In [ ]:
df_tareas.info()

In [28]:
columnas_tareas =  df_tareas.columns.tolist()
columnas_tareas

['id', 'msf_attribute_1__c', 'msf_attribute_2__c', 'msf_attribute_3__c', 'msf_attribute_4__c', 'msf_attribute_5__c', 'msf_campaigndonationreporting__c', 'msf_campaignentryreporting__c', 'msf_canalsalidaconcatenado__c', 'msf_isemergency__c', 'msf_isonline__c', 'msf_objective__c', 'msf_objectivepublic__c', 'msf_outboundchannel1__c', 'msf_outboundchannel2__c', 'msf_ownby__c', 'msf_previousstepchannel__c', 'msf_promoterindividual__c', 'msf_provider__c', 'msf_segment__c', 'msf_thematic__c', 'ownerid', 'recordtypeid', 'status']


In [29]:
# Se revisa el nº total de registros y columnas de la tabla "tareas"
df_tareas.shape

(11501, 24)


In [30]:
# Se analizan la cantidad de nulos en cada variable del dataset
nulos = df_tareas.isnull().sum()
nulos

id                                  0
msf_attribute_1__c                  0
msf_attribute_2__c                  0
msf_attribute_3__c                  0
msf_attribute_4__c                  0
msf_attribute_5__c                  0
msf_campaigndonationreporting__c    0
msf_campaignentryreporting__c       0
msf_canalsalidaconcatenado__c       0
msf_isemergency__c                  0
msf_isonline__c                     0
msf_objective__c                    0
msf_objectivepublic__c              0
msf_outboundchannel1__c             0
msf_outboundchannel2__c             0
msf_ownby__c                        0
msf_previousstepchannel__c          0
msf_promoterindividual__c           0
msf_provider__c                     0
msf_segment__c                      0
msf_thematic__c                     0
ownerid                             0
recordtypeid                        0
status                              0
dtype: int64


In [31]:
# Comprobamos si existen duplicados
print(len(df_tareas))
print(len(df_tareas.drop_duplicates()))

11501
11501


<div class="alert alert-block alert-success">
No existen duplicados en el dataset
</div>

## Se va a analizar cada variable por tablas

<div class="alert alert-block alert-warning">
<b>Definición de funciones para el analisis exploratorio del dataset:</b> <br>
            -> Se definirá la función conunt_nulos, que servirá para contar el nº de vacios y nulos de las variables del dataset. <br>
            ->  Se definirá la función grafica_categorica, que servirá para plotear la distribucioes de las variables categoricas del dataset.<br>
            ->  Se definirá la función freq_variables, que se utilizará para realizar un conteo de cada psoible valor de la variable.<br>
</div>

In [8]:
# Contabilizacion de los nulos valores de la variable
def count_nulos(df,variable,list_delete):
    
    '''
    Función count_nulos

    Uso:
    Sirva para realizar un conteo de los registros nulos y vacios de una variable seleccionada del dataframe de entrada.

    Parametros entrada:
    - df : dataframe de entrada.
    - variable: nombre de la variable sobre la que se quiere graficar. Variable tipo categorica o numerica.

    Salida:
    Frase en la que se detectará el nº de nulos y vacios, asi como su porcentaje del total de la tabla.
        
    '''
        
    nulos = df[[variable]].isnull().sum()[0]
    vacios = (df[[var]] == '').sum(axis=0)[0]
    print(f"El nº de nulos para la variable {variable} es {nulos}. Lo que supone un {(nulos/df.shape[0])*100}%")
    print(f"El nº de vacios para la variable {var} es {vacios}. Lo que supone un {(vacios/df.shape[0])*100}%")
    
    if ((nulos + vacios) / df.shape[0]) *100 > 20:
        list_delete.append(var)
        return list_delete

In [9]:
# Contabilizacion de los posibles valores de la variable
def freq_variables(df,variable):
    
    '''
    Función freq_variables

    Uso:
    Sirva para realizar un conteo de los posibles valores de una variable seleccionada del dataframe de entrada.

    Parametros entrada:
    - df : dataframe de entrada.
    - variable: nombre de la variable sobre la que se quiere graficar. Variable tipo categorica o numerica.

    Salida:
    La salida es una tabla que contiene los posibles valores de la variable de entrada y el nº de registros de cada valor en el dataframe.
        
    '''
    afreq = df[variable].value_counts()
    pfreq = df[variable].value_counts(normalize = True)*100
    freq_report = pd.DataFrame({'# Tot':afreq, '% Tot':pfreq})
    return freq_report

## 1. Tabla recurring donation

In [8]:
# Vamos a analizar la tabla recurring donation
df = df_re_donation

In [9]:
# Se crea una lista por ahora vacia, en la que se irán añadiendo las variables que se van a eliminar del dataset por motivos varios: no utilidad, gran volumen de nulos, ...
col_to_delete_re_donation=list()

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> IsDeleted: Variable booleana <br>
</div>

In [10]:
# Vamos a realizar analisis por cada variable
var = "isdeleted"

In [11]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable isdeleted es 0. Lo que supone un 0.0%
El nº de vacios para la variable isdeleted es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [12]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

         # Tot  % Tot
False  1198207  100.0


<div class="alert alert-block alert-success">
<b>isdeleted: Sirve para descartar registros que hayan sido errores en caso de que tome valor true.</b> <br>
No hay ningun true por lo que se pueden considerar todos los registros. Se puede eliminar la variable al no aportar más valor. 
</div>

In [13]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_re_donation.append(var)
col_to_delete_re_donation

['isdeleted']


<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_AnnualizedQuota__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_annualizedquota__c"

In [15]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable msf_annualizedquota__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_annualizedquota__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [16]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

             # Tot      % Tot
120.00      285404  23.819257
60.00       129853  10.837276
180.00      122956  10.261666
240.00       76435   6.379115
144.00       57831   4.826462
72.00        48674   4.062236
36.00        30804   2.570841
360.00       28630   2.389404
300.00       26549   2.215727
96.00        21951   1.831987
72.12        21267   1.774902
0.00         19245   1.606150
84.00        16695   1.393332
100.00       15786   1.317469
168.00       13694   1.142874
40.00        12404   1.035213
20.00        11682   0.974957
50.00        11473   0.957514
80.00         9212   0.768815
600.00        8751   0.730341
30.00         8515   0.710645
200.00        8126   0.678180
10.00         8036   0.670669
48.00         7544   0.629607
204.00        7497   0.625685
51.96         6882   0.574358
480.00        6742   0.562674
216.00        6595   0.550406
12.00         6240   0.520778
150.00        6048   0.504754
132.00        5844   0.487729
60.10         5371   0.448253
108.00    

<div class="alert alert-block alert-success">
<b>msf_annualizedquota__c: Aportación del socio anualizada.</b> <br>
No hay vacios ni nulos, se acumula casi el 50% de la poblacion entre los 60 y los 180€. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_cancelationdate__c: Variable fecha <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "msf_cancelationdate__c"

In [22]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable msf_cancelationdate__c es 483006. Lo que supone un 40.3107309504952%
El nº de vacios para la variable msf_cancelationdate__c es 0. Lo que supone un 0.0%
['isdeleted', 'msf_cancelationdate__c']


In [21]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

            # Tot     % Tot
2014-03-13   2887  0.403663
2020-03-12   2637  0.368708
2018-03-07   2198  0.307326
2018-04-09   1957  0.273629
2023-05-10   1867  0.261045
2013-08-09   1623  0.226929
2016-05-09   1546  0.216163
2022-02-07   1491  0.208473
2014-02-07   1456  0.203579
2023-01-04   1390  0.194351
2021-12-06   1380  0.192953
2020-02-06   1352  0.189038
2019-11-08   1348  0.188478
2022-10-05   1334  0.186521
2022-12-05   1320  0.184564
2022-11-04   1319  0.184424
2020-08-05   1318  0.184284
2019-10-07   1313  0.183585
2009-03-18   1308  0.182886
2019-12-06   1301  0.181907
2020-07-03   1292  0.180649
2021-05-06   1277  0.178551
2009-02-18   1268  0.177293
2021-03-04   1265  0.176873
2020-04-06   1258  0.175895
2017-06-08   1256  0.175615
2017-01-09   1234  0.172539
2013-11-08   1230  0.171980
2023-02-03   1222  0.170861
2017-09-05   1217  0.170162
2017-10-05   1206  0.168624
2016-10-10   1205  0.168484
2023-04-05   1197  0.167366
2019-01-07   1195  0.167086
2014-10-08   1191  0

<div class="alert alert-block alert-success">
<b>msf_cancelationdate__c: Fecha de cancelación de la aportación.</b> <br>
Exite un 40% de vacios, que son las donaciones actualmente en vigor.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_cancelationreason__c: Variable alfanumerica<br>
</div>

In [23]:
# Vamos a realizar analisis por cada variable
var = "msf_cancelationreason__c"

In [24]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable msf_cancelationreason__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_cancelationreason__c es 483501. Lo que supone un 40.35204267710004%
['isdeleted', 'msf_cancelationdate__c', 'msf_cancelationreason__c']


In [25]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                                                     # Tot      % Tot
                                                    483501  40.352043
Impago aportaciones                                 285429  23.821343
Unpaid                                              102935   8.590753
Baja económica argumentada                           66594   5.557804
3 Obs/Tcs Devueltas                                  66286   5.532099
Económico                                            39385   3.286995
N/S                                                  35770   2.985294
Razones personales                                   34347   2.866533
Voluntary withdrawal                                 16894   1.409940
Deceased                                             15218   1.270064
Otras razones                                         8112   0.677012
2 OBS devueltas es la primera de su historia          5346   0.446167
Baja argumentada                                      4854   0.405105
Baja por cambio de t

<div class="alert alert-block alert-success">
<b>msf_cancelationreason__c: Motivo de cancelación de la aportación.</b> <br>
Exite un 40% de vacios, que son las donaciones actualmente en vigor. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_currentcampaign__c: Variable alfanumerica<br>
</div>

In [26]:
# Vamos a realizar analisis por cada variable
var = "msf_currentcampaign__c"

In [27]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable msf_currentcampaign__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_currentcampaign__c es 19006. Lo que supone un 1.5862033855585889%


In [28]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                    # Tot     % Tot
7013Y000001mqtMQAQ  95523  7.972162
7013Y000001mqtnQAA  47771  3.986874
7013Y000001vXGdQAM  30352  2.533118
7013Y000001mrBLQAY  30029  2.506161
7013Y000001mrCzQAI  28901  2.412021
7013Y000001mqttQAA  25847  2.157140
7013Y000001mr4CQAQ  25713  2.145956
7013Y000001mr2DQAQ  22950  1.915362
7013Y000001mqtuQAA  21700  1.811039
7013Y000001mquNQAQ  21452  1.790342
7013Y000001mr2cQAA  19578  1.633941
7013Y000001mqtIQAQ  19453  1.623509
                    19006  1.586203
7013Y000001mr25QAA  18318  1.528784
7013Y000001n865QAA  16306  1.360867
7013Y000001mrBSQAY  15800  1.318637
7013Y000001mr4AQAQ  15567  1.299191
7013Y000001mr1MQAQ  14609  1.219238
7013Y000001mrBoQAI  14398  1.201629
7013Y000001n860QAA  14002  1.168579
7013Y000001mr1mQAA  13513  1.127768
7013Y000001mN98QAE  13499  1.126600
7013Y000001mqtCQAQ  12964  1.081950
7013Y000001vXGiQAM  12663  1.056829
7013Y000001DrxyQAC  10916  0.911028
7013Y000001mqtPQAQ  10817  0.902766
7013Y000001mr1sQAA   9728  0

<div class="alert alert-block alert-success">
<b>msf_currentcampaign__c: Campaña actual de la aportación.</b> <br>
Exite un 1,5% de vacios. Hay muchisima distribución entre los diferentes id de campaña.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_currentleadsource1__c: Variable alfanumerica<br>
</div>

In [29]:
# Vamos a realizar analisis por cada variable
var = "msf_currentleadsource1__c"

In [30]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable msf_currentleadsource1__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_currentleadsource1__c es 19170. Lo que supone un 1.5998905030599888%


In [31]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                                 # Tot      % Tot
Telemarketing                   509871  42.552831
Persona a persona               255845  21.352321
Cupón                            84961   7.090678
Otro                             71618   5.977097
Personal con tablet              64333   5.369106
Web MSF                          63379   5.289487
Teléfono campaña                 48761   4.069497
Teléfono SAS                     21594   1.802193
Web campaña                      20290   1.693364
                                 19170   1.599891
Entidad financiera                8710   0.726919
Email a SAS                       8083   0.674591
Web terceros                      7062   0.589381
Teléfono web                      7001   0.584290
Correo postal sin cupón           4185   0.349272
Web MSF Mi perfil                 1057   0.088215
Cloud page                         885   0.073860
Eventos                            763   0.063678
Teléfono Officers                  389   0.032465


<div class="alert alert-block alert-success">
<b>msf_currentleadsource1__c: Canal actual de la donación.</b> <br>
Exite un 1,5% de vacios. La mayoria es telemarketing o persona a persona.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_leadsource1__c: Variable alfanumerica<br>
</div>

In [32]:
# Vamos a realizar analisis por cada variable
var = "msf_leadsource1__c"

In [33]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable msf_leadsource1__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_leadsource1__c es 173. Lo que supone un 0.014438239803306108%


In [34]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                                 # Tot      % Tot
Persona a persona               374964  31.293758
Telemarketing                   289168  24.133393
Otro                            136249  11.371074
Cupón                           133197  11.116360
Web MSF                          76674   6.399061
Personal con tablet              74059   6.180819
Teléfono campaña                 61869   5.163465
Web campaña                      14210   1.185939
Web terceros                     12698   1.059750
Teléfono SAS                      6595   0.550406
Teléfono web                      5748   0.479717
Email a SAS                       5354   0.446834
Correo postal sin cupón           5008   0.417958
Eventos                           1577   0.131613
Entidad financiera                 343   0.028626
                                   173   0.014438
Email a Empresas                   162   0.013520
Teléfono Officers                  126   0.010516
Email a officers Mid Donors          9   0.000751


<div class="alert alert-block alert-success">
<b>msf_leadsource1__c: Canal de captura de la donación.</b> <br>
Exite menos del 1% de vacios. La mayoria es telemarketing o persona a persona.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npe03__amount__c: Variable numerica<br>
</div>

In [37]:
# Vamos a realizar analisis por cada variable
var = "npe03__amount__c"

In [38]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npe03__amount__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npe03__amount__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [39]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

             # Tot      % Tot
10.00       283542  23.663858
15.00       133048  11.103924
5.00        113650   9.485006
20.00        89248   7.448463
12.00        62605   5.224890
30.00        50617   4.224395
6.00         48472   4.045378
3.00         30398   2.536957
25.00        29725   2.480790
50.00        24828   2.072096
8.00         22023   1.837996
60.00        19257   1.607151
0.00         19245   1.606150
7.00         16418   1.370214
6.01         16086   1.342506
100.00       14742   1.230338
14.00        13508   1.127351
40.00        12300   1.026534
30.05        12251   1.022444
18.03        11212   0.935731
18.00         8598   0.717572
17.00         7785   0.649721
60.10         7704   0.642961
4.33          6937   0.578948
35.00         6936   0.578865
9.00          6883   0.574442
150.00        6295   0.525368
4.00          6150   0.513267
120.00        6065   0.506173
11.00         5674   0.473541
16.00         5324   0.444331
13.00         5227   0.436235
22.00     

<div class="alert alert-block alert-success">
<b>npe03__amount__c: Importe de la donación.</b> <br>
No exiten vacios, hay mucha distribución entre diferentes importes.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npe03__date_established__c: Variable fecha<br>
</div>

In [40]:
# Vamos a realizar analisis por cada variable
var = "npe03__date_established__c"

In [41]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npe03__date_established__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npe03__date_established__c es 0. Lo que supone un 0.0%


In [42]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

            # Tot     % Tot
2004-01-01   5000  0.417290
2000-02-01   4604  0.384241
1994-10-01   3829  0.319561
2000-01-01   3810  0.317975
1995-02-01   3377  0.281838
2004-02-01   2851  0.237939
1998-03-01   2839  0.236937
1994-02-01   2801  0.233766
1999-01-01   2739  0.228592
1999-02-01   2234  0.186445
1996-02-01   2030  0.169420
1994-07-01   1820  0.151894
2001-01-01   1754  0.146385
2013-08-28   1740  0.145217
1997-02-01   1674  0.139709
2001-02-01   1567  0.130779
1998-02-01   1560  0.130195
2000-03-01   1484  0.123852
1994-01-01   1466  0.122349
1995-04-01   1464  0.122183
1998-01-01   1438  0.120013
2015-02-25   1437  0.119929
1992-11-01   1394  0.116340
2016-07-28   1394  0.116340
2001-12-31   1391  0.116090
2005-02-02   1384  0.115506
1994-09-01   1382  0.115339
2014-12-23   1369  0.114254
2005-02-03   1357  0.113253
2014-05-23   1345  0.112251
2014-04-30   1340  0.111834
2015-02-19   1326  0.110665
2017-04-28   1325  0.110582
2014-11-26   1309  0.109247
2015-09-30   1300  0

<div class="alert alert-block alert-success">
<b>npe03__date_established__c: Fecha de alta de la donación recurrente.</b> <br>
No exiten vacios, hay mucha distribución, se puede utilizar para la generacion de una variable de "meses_desde_inicio", considerando que alguno de ellos ha cancelado.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npe03__installment_period__c: Variable numerica <br>
</div>

In [17]:
# Vamos a realizar analisis por cada variable
var = "npe03__installment_period__c"

In [18]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npe03__installment_period__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npe03__installment_period__c es 28832. Lo que supone un 2.4062620231729577%


In [19]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

            # Tot      % Tot
Monthly    928070  77.454897
Yearly     125456  10.470311
Quarterly   89401   7.461232
            28832   2.406262
Semestral   20047   1.673083
Bimensual    6401   0.534215


<div class="alert alert-block alert-success">
<b>npe03__installment_period__c: Periodicidad de las aportaciones.</b> <br>
Exite un 2% de vacios. Lo más comun es tener cuota mensual.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npe03__open_ended_status__c: Variable numerica <br>
</div>

In [43]:
# Vamos a realizar analisis por cada variable
var = "npe03__open_ended_status__c"

In [44]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npe03__open_ended_status__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npe03__open_ended_status__c es 0. Lo que supone un 0.0%


In [45]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

         # Tot      % Tot
Closed  715235  59.692107
Open    482947  40.305807
None        24   0.002003
Close        1   0.000083


<div class="alert alert-block alert-success">
<b>npe03__open_ended_status__c: Indicador de donacion recurrente cancelada.</b> <br>
Exiten unos pocos registros erroneos, pero la variable es util para saber quienes son los activos a dia de hoy que serán el objetivo y los cancelados podran ser utilizados como datos historicos en el modelo pero a dia de hoy no es objetivo.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npe03__paid_amount__c: Variable numerica<br>
</div>

In [49]:
# Vamos a realizar analisis por cada variable
var = "npe03__paid_amount__c"

In [50]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npe03__paid_amount__c es 1899. Lo que supone un 0.15848680570218668%
El nº de vacios para la variable npe03__paid_amount__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [51]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

           # Tot      % Tot
0.00      465514  38.912554
670.00     40774   3.408320
1005.00    22332   1.866743
1340.00    14305   1.195762
335.00     11733   0.980767
60.00      11494   0.960789
30.00      11422   0.954771
10.00       9926   0.829719
20.00       9869   0.824955
804.00      9857   0.823952
120.00      8634   0.721720
300.00      8157   0.681848
15.00       7822   0.653845
40.00       7377   0.616647
402.00      6984   0.583796
180.00      6601   0.551781
90.00       6463   0.540245
50.00       6400   0.534979
100.00      6311   0.527540
150.00      6143   0.513497
2010.00     6135   0.512828
360.00      6074   0.507729
600.00      6061   0.506642
240.00      5297   0.442779
660.00      5180   0.432999
80.00       4977   0.416030
1675.00     4911   0.410513
330.00      4385   0.366544
45.00       4376   0.365792
200.00      4268   0.356764
5.00        4123   0.344644
70.00       3990   0.333526
12.00       3795   0.317226
75.00       3642   0.304437
210.00      3537   0

<div class="alert alert-block alert-success">
<b>npe03__paid_amount__c: Importe acumulado de todas las donaciones hasta el momento.</b> <br>
Existen algunos nulos, hay mucha distribución entre diferentes importes.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npe03__recurring_donation_campaign__c: Variable alfanumerica<br>
</div>

In [46]:
# Vamos a realizar analisis por cada variable
var = "npe03__recurring_donation_campaign__c"

In [47]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npe03__recurring_donation_campaign__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npe03__recurring_donation_campaign__c es 1. Lo que supone un 8.3458033545122e-05%


In [48]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                    # Tot     % Tot
7013Y000001mqtnQAA  55336  4.618234
7013Y000001mr4CQAQ  38368  3.202118
7013Y000001mrCzQAI  35953  3.000567
7013Y000001mr2DQAQ  32217  2.688767
7013Y000001mr2cQAA  27436  2.289755
7013Y000001mrBSQAY  24330  2.030534
7013Y000001mr1MQAQ  24203  2.019935
7013Y000001mr25QAA  23769  1.983714
7013Y000001mr4AQAQ  23699  1.977872
7013Y000001mrBoQAI  22041  1.839499
7013Y000001mr1mQAA  18450  1.539801
7013Y000001mr2CQAQ  14340  1.196788
7013Y000001mqvKQAQ  13619  1.136615
7013Y000001mr2TQAQ  13221  1.103399
7013Y000001mr1sQAA  12380  1.033210
7013Y000001mquNQAQ  12215  1.019440
7013Y000001mqtPQAQ  11558  0.964608
7013Y000001mqq8QAA  11367  0.948667
7013Y000001mrC7QAI  10557  0.881066
7013Y000001mr2RQAQ  10379  0.866211
7013Y000001mqtmQAA  10306  0.860118
7013Y000001mr3eQAA   9847  0.821811
7013Y000001mr2tQAA   9588  0.800196
7013Y000001mr2KQAQ   9525  0.794938
7013Y000001mqtQQAQ   9519  0.794437
7013Y000001mrBhQAI   9304  0.776494
7013Y000001mr2sQAA   9188  0

<div class="alert alert-block alert-success">
<b>npe03__recurring_donation_campaign__c: Campaña de camptura de la aportación.</b> <br>
Exite muy pocos vacios. Hay muchisima distribución entre los diferentes id de campaña.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npe03__total_paid_installments__c: Variable numerica<br>
</div>

In [52]:
# Vamos a realizar analisis por cada variable
var = "npe03__total_paid_installments__c"

In [53]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npe03__total_paid_installments__c es 1899. Lo que supone un 0.15848680570218668%
El nº de vacios para la variable npe03__total_paid_installments__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [54]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

        # Tot      % Tot
0.0    465514  38.912554
67.0   240168  20.075766
6.0     45376   3.793003
1.0     34577   2.890309
5.0     32738   2.736586
2.0     26558   2.219997
22.0    22576   1.887139
3.0     20987   1.754314
4.0     18250   1.525527
23.0    17528   1.465175
11.0    11887   0.993640
7.0     10423   0.871264
8.0     10076   0.842258
12.0     9527   0.796367
9.0      9273   0.775135
10.0     8725   0.729327
13.0     8149   0.681179
16.0     6981   0.583545
14.0     6895   0.576357
20.0     6394   0.534478
19.0     6183   0.516840
15.0     6163   0.515168
21.0     6139   0.513162
17.0     5729   0.478890
18.0     5632   0.470782
33.0     5537   0.462841
25.0     4887   0.408507
24.0     4797   0.400984
27.0     4743   0.396470
34.0     4673   0.390618
26.0     4623   0.386439
66.0     4569   0.381925
28.0     4163   0.347987
31.0     4105   0.343139
29.0     4099   0.342638
36.0     4006   0.334864
30.0     3949   0.330099
42.0     3881   0.324415
32.0     3729   0.311709


<div class="alert alert-block alert-success">
<b>npe03__total_paid_installments__c: Numero de cuotas pagadas.</b> <br>
Exite muy pocos vacios, pero hay un 39% con 0 cuotas.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp4hub__payment_method__c: Variable alfanumerica<br>
</div>

In [55]:
# Vamos a realizar analisis por cada variable
var = "npsp4hub__payment_method__c"

In [56]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_re_donation)

El nº de nulos para la variable npsp4hub__payment_method__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npsp4hub__payment_method__c es 19192. Lo que supone un 1.6017265797979814%


In [57]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                # Tot      % Tot
Direct Debit  1172815  97.880834
                19192   1.601727
CreditCard       6124   0.511097
ACMA               76   0.006343


<div class="alert alert-block alert-success">
<b>npsp4hub__payment_method__c: Forma de pago de la aportación.</b> <br>
Exite muy pocos vacios. Pero practicamente todas las aportaciones con con debito directo, por lo que la variab le no aporta valor y se eliminará.
</div>

In [61]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_re_donation.append(var)
col_to_delete_re_donation

['isdeleted', 'msf_cancelationdate__c', 'msf_cancelationreason__c', 'npsp4hub__payment_method__c', 'npsp4hub__payment_method__c']


## 2. Tabla modificacion de cuota

In [62]:
# Vamos a analizar la tabla recurring donation
df = df_mod_cuota

In [78]:
# Se crea una lista por ahora vacia, en la que se irán añadiendo las variables que se van a eliminar del dataset por motivos varios: no utilidad, gran volumen de nulos, ...
col_to_delete_mod_cuota=list()

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> name: Variable string <br>
</div>

In [79]:
# Vamos a realizar analisis por cada variable
var = "name"

In [65]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable name es 0. Lo que supone un 0.0%
El nº de vacios para la variable name es 0. Lo que supone un 0.0%


In [66]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

ERROR: output size exceeded 6291456 bytes


<div class="alert alert-block alert-success">
<b>name: Codigo de la cuota modificada.</b> <br>
No hay vacios. Se puede extraer la primera letra de cada nombre para tener un indicador de si se ha modificado la cuota o no, y si se ha hecho ha sido incremento o decremento. Pero esta informacion ya está en otra variable (msf_changetype__c) por lo que no aporta valor y se eliminará.
    No olvidarnos de que el objetivo del analisis es el aumento de cuotas, por lo que la muestra representativa para el modelo será la que haya tenido modificaciones incrementales.
</div>

In [80]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_mod_cuota.append(var)
col_to_delete_mod_cuota

['name']


<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_changeamount__c: Variable numerica <br>
</div>

In [67]:
# Vamos a realizar analisis por cada variable
var = "msf_changeamount__c"

In [68]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_changeamount__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_changeamount__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [69]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

               # Tot      % Tot
1.000000e+01  407031  20.320876
5.000000e+00  294167  14.686181
1.500000e+01  152100   7.593538
2.000000e+00  133031   6.641525
3.000000e+00  121177   6.049718
2.000000e+01  101458   5.065254
6.000000e+00   85147   4.250933
4.000000e+00   69406   3.465069
1.200000e+01   65735   3.281796
3.000000e+01   58719   2.931525
1.000000e+00   43861   2.189745
5.000000e+01   33425   1.668731
2.500000e+01   30875   1.541423
8.000000e+00   29716   1.483561
6.010000e+00   28716   1.433636
6.000000e+01   22945   1.145521
7.000000e+00   22941   1.145321
3.005000e+01   22602   1.128397
0.000000e+00   19575   0.977275
1.803000e+01   18083   0.902787
1.000000e+02   16167   0.807132
6.010000e+01   12438   0.620963
4.330000e+00   12111   0.604637
4.000000e+01   11849   0.591557
1.202000e+01   11641   0.581173
9.000000e+00    7611   0.379976
1.800000e+01    6958   0.347376
1.200000e+02    6128   0.305938
1.100000e+01    5513   0.275235
3.990000e+00    4887   0.243982
1.000000

<div class="alert alert-block alert-success">
<b>msf_changeamount__c: Importe del cambio de cuota.</b> <br>
No hay vacios. sobre el 35% es de 5 a 10€ de cambio.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_changeannualizedquota__c: Variable numerica <br>
</div>

In [70]:
# Vamos a realizar analisis por cada variable
var = "msf_changeannualizedquota__c"

In [71]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_changeannualizedquota__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_changeannualizedquota__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [72]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

               # Tot      % Tot
1.200000e+02  385981  19.269962
6.000000e+01  301818  15.068155
2.400000e+01  133909   6.685358
1.800000e+02  132844   6.632189
3.600000e+01  119288   5.955410
7.200000e+01   82678   4.127669
2.400000e+02   70913   3.540306
4.800000e+01   70385   3.513946
1.200000e+01   56315   2.811506
1.440000e+02   55521   2.771866
2.000000e+01   40328   2.013361
7.212000e+01   36336   1.814062
4.000000e+01   30008   1.498139
9.600000e+01   27670   1.381415
3.600000e+02   24139   1.205131
5.000000e+01   23340   1.165241
1.000000e+01   23082   1.152361
1.000000e+02   20620   1.029446
8.400000e+01   20236   1.010275
3.000000e+02   19774   0.987210
3.000000e+01   18265   0.911874
8.000000e+01   13937   0.695800
5.196000e+01   12108   0.604488
3.005000e+01   11513   0.574782
6.010000e+01    9995   0.498997
1.202000e+02    7670   0.382922
5.000000e+00    7670   0.382922
2.000000e+02    7287   0.363801
2.500000e+01    7230   0.360955
1.442400e+02    7188   0.358858
1.500000

<div class="alert alert-block alert-success">
<b>msf_changeannualizedquota__c: Importe de la modificacion en terminos anuales.</b> <br>
No hay vacios.Quizá es más interesante que la mensual, ya que el otro es por cuota que no todos tienen la misma periodicidad, en cambio esta es comparable para todos los registros. Por lo que no se va a tener en cuenta la anterior msf_changeamount__c.
</div>

In [81]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_mod_cuota.append("msf_changeamount__c")
col_to_delete_mod_cuota

['name', 'msf_changeamount__c']


<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_changetype__c: Variable string <br>
</div>

In [82]:
# Vamos a realizar analisis por cada variable
var = "msf_changetype__c"

In [83]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_changetype__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_changetype__c es 0. Lo que supone un 0.0%


In [84]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                                            # Tot      % Tot
Initial                                   1174130  58.618016
Increase                                   767916  38.337929
Decrease                                    60927   3.041758
Changes_without_variation_annualized_fee       46   0.002297


<div class="alert alert-block alert-success">
<b>col_to_delete_mod_cuota: Clase de modificación.</b> <br>
No hay vacios. Como el objetivo del analisis es el aumento de cuotas, por lo que la muestra representativa para el modelo será la que haya tenido modificaciones incrementales. Por lo que a la hora de crear el dataset final, esta variable nos servirá para crear una variable booleana que determinará como 1 a los que hayan incrementado y 0 al resto. Siendo esta variable a crear la variable objetivo del modelo.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_leadsource1__c: Variable string <br>
</div>

In [85]:
# Vamos a realizar analisis por cada variable
var = "msf_leadsource1__c"

In [86]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_leadsource1__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_leadsource1__c es 324788. Lo que supone un 16.21492357286676%


In [87]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                                 # Tot      % Tot
Telemarketing                   832085  41.541543
Persona a persona               371059  18.524987
                                324788  16.214924
Web MSF                         105789   5.281478
Cupón                            86079   4.297463
Teléfono campaña                 80095   3.998714
Personal con tablet              74862   3.737458
Entidad financiera               67438   3.366818
Teléfono SAS                     17534   0.875379
Teléfono web                     12659   0.631996
Web terceros                     12565   0.627303
Email a SAS                      10538   0.526106
Web campaña                       4192   0.209284
Web MSF Mi perfil                  939   0.046879
Cloud page                         922   0.046031
Otro                               542   0.027059
Teléfono Officers                  441   0.022017
Correo postal sin cupón            227   0.011333
Email a Empresas                   207   0.010334


<div class="alert alert-block alert-success">
<b>msf_leadsource1__c: Canal de modificación.</b> <br>
Hay un 16% de vacios. La distribucion se concentra en telemarketing.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_leadsource2__c: Variable string <br>
</div>

In [88]:
# Vamos a realizar analisis por cada variable
var = "msf_leadsource2__c"

In [89]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_leadsource2__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_leadsource2__c es 324964. Lo que supone un 16.22371030928813%


In [90]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                               # Tot      % Tot
Telemarketing                 832029  41.538747
Persona a persona             445916  22.262195
                              324964  16.223710
Formulario web                124405   6.210875
Cupón                          86069   4.296964
Teléfono campaña               80020   3.994970
Entidad financiera             67431   3.366468
Teléfonos SAS                  17534   0.875379
Teléfono web                   12654   0.631746
Email                          10779   0.538138
Otro                             541   0.027009
Teléfonos Officers               441   0.022017
Correo postal sin cupón          227   0.011333
Teléfono Herencias y Legados       6   0.000300
SMS                                2   0.000100
Redes Sociales                     1   0.000050


<div class="alert alert-block alert-success">
<b>msf_leadsource2__c: Canal de modificación agrupación 2.</b> <br>
Hay un 16% de vacios. La distribucion se concentra en telemarketing.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_leadsource3__c: Variable string <br>
</div>

In [91]:
# Vamos a realizar analisis por cada variable
var = "msf_leadsource3__c"

In [92]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_leadsource3__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_leadsource3__c es 324964. Lo que supone un 16.22371030928813%


In [93]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                     # Tot      % Tot
Teléfono            942686  47.063258
Personal            445916  22.262195
                    324964  16.223710
Online              135185   6.749062
Correo postal        86296   4.308297
Entidad financiera   67431   3.366468
Otro                   541   0.027009


<div class="alert alert-block alert-success">
<b>msf_leadsource3__c: Canal de modificación agrpacion 3.</b> <br>
Hay un 16% de vacios. La distribucion se concentra en telemarketing. Esta variable más agrupada puede ser mejor para el analisis que las anteriores.
</div>

In [ ]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_mod_cuota.append("msf_leadsource1__c")
col_to_delete_mod_cuota.append("msf_leadsource2__c")
col_to_delete_mod_cuota

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_newamount__c: Variable numerica <br>
</div>

In [96]:
# Vamos a realizar analisis por cada variable
var = "msf_newamount__c"

In [97]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_newamount__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_newamount__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [98]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

               # Tot      % Tot
1.000000e+01  397732  19.856626
1.500000e+01  214087  10.688216
2.000000e+01  156745   7.825438
5.000000e+00  152635   7.620247
1.200000e+01  108041   5.393908
3.000000e+01   86873   4.337103
6.000000e+00   74110   3.699915
2.500000e+01   57170   2.854192
5.000000e+01   43554   2.174418
8.000000e+00   42810   2.137274
3.000000e+00   38335   1.913861
1.400000e+01   37487   1.871525
6.000000e+01   34637   1.729240
7.000000e+00   32136   1.604378
4.000000e+01   27600   1.377920
1.000000e+02   26747   1.335334
6.010000e+00   25495   1.272829
3.005000e+01   23269   1.161696
1.800000e+01   20395   1.018213
1.700000e+01   19095   0.953311
1.803000e+01   19066   0.951863
3.500000e+01   16640   0.830746
6.010000e+01   15309   0.764296
9.000000e+00   15052   0.751466
1.300000e+01   14533   0.725555
1.500000e+02   12377   0.617917
4.330000e+00   12127   0.605436
1.100000e+01   11957   0.596949
1.200000e+02   11612   0.579725
2.600000e+01   11549   0.576580
2.200000

<div class="alert alert-block alert-success">
<b>msf_newamount__c: Nueva cuota.</b> <br>
No hay vacios. Muy distribuida.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_newannualizedquota__c: Variable numerica <br>
</div>

In [99]:
# Vamos a realizar analisis por cada variable
var = "msf_newannualizedquota__c"

In [100]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_newannualizedquota__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_newannualizedquota__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [101]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

               # Tot      % Tot
1.200000e+02  411248  20.531408
1.800000e+02  201969  10.083229
6.000000e+01  183545   9.163418
2.400000e+02  139200   6.949510
1.440000e+02  102569   5.120720
7.200000e+01   76466   3.817537
3.000000e+02   51677   2.579956
3.600000e+02   51534   2.572816
9.600000e+01   43025   2.148008
3.600000e+01   40100   2.001978
1.680000e+02   38432   1.918704
7.212000e+01   35849   1.789748
8.400000e+01   33843   1.689600
1.000000e+02   29554   1.475473
5.000000e+01   19528   0.974928
4.000000e+01   19125   0.954809
2.040000e+02   18343   0.915768
8.000000e+01   17256   0.861500
2.160000e+02   16975   0.847471
2.000000e+02   16785   0.837985
2.000000e+01   15636   0.780622
4.800000e+02   15150   0.756358
6.000000e+02   15139   0.755809
1.560000e+02   14188   0.708331
1.080000e+02   13249   0.661452
3.000000e+01   13134   0.655710
1.320000e+02   12653   0.631696
4.800000e+01   12429   0.620513
5.196000e+01   12115   0.604837
1.500000e+02   11843   0.591257
3.120000

<div class="alert alert-block alert-success">
<b>msf_newannualizedquota__c: Nueva cuota anualizada.</b> <br>
No hay vacios. Muy distribuida.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_changedate__c: Variable fecha <br>
</div>

In [102]:
# Vamos a realizar analisis por cada variable
var = "msf_changedate__c"

In [103]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_mod_cuota)

El nº de nulos para la variable msf_changedate__c es 186. Lo que supone un 0.00928598280894989%
El nº de vacios para la variable msf_changedate__c es 0. Lo que supone un 0.0%


In [104]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

            # Tot     % Tot
2018-01-03  15448  0.771307
2017-02-02  13291  0.663610
2017-12-04  12659  0.632055
2018-02-01  11043  0.551369
2014-12-02  10917  0.545078
2017-01-02  10121  0.505334
2016-01-04   9366  0.467638
2018-11-02   8817  0.440226
2015-11-03   8623  0.430540
2017-11-02   8302  0.414513
2015-12-02   7241  0.361538
2008-06-02   7211  0.360040
2018-12-03   7191  0.359041
2015-02-02   6979  0.348456
2018-08-01   6805  0.339769
2014-11-03   6682  0.333627
2018-10-02   6646  0.331830
2020-01-02   6592  0.329134
2016-02-01   6443  0.321694
2016-12-01   6211  0.310111
2017-03-02   6090  0.304069
2016-03-01   5758  0.287493
2010-12-02   5500  0.274611
2018-09-03   5452  0.272214
2013-01-02   5432  0.271216
2008-07-04   5354  0.267321
2015-05-04   5306  0.264925
2020-07-01   5294  0.264326
2015-04-01   5174  0.258334
2019-11-04   5094  0.254340
2011-06-01   5079  0.253591
2000-02-01   5079  0.253591
2015-03-02   5033  0.251294
2019-03-01   4957  0.247499
2019-01-02   4899  0

<div class="alert alert-block alert-success">
<b>msf_changedate__c: Fecha de modificacion de la cuota.</b> <br>
Hay 186 vacios. Se puede utilizar para saber desde hace cuanto no se modifica la cuota, y priorizar a la hpra de seleccionar registros para el modelo y que sea muestra reciente.
</div>

## 3. Tabla contactos

In [10]:
# Se analizar solo los contactos que tienen donaciones recurrentes, ya que son el objetivo del analisis, por ello solo se tendrán en cuanta los contactos coincidentes entre la tabla recurring donation y contactos
df_contactos_f = df_contactos[df_contactos.id.isin(df_re_donation.npe03__contact__c)]

In [11]:
# Vamos a analizar la tabla recurring donation
df=df_contactos_f

In [12]:
# Se crea una lista por ahora vacia, en la que se irán añadiendo las variables que se van a eliminar del dataset por motivos varios: no utilidad, gran volumen de nulos, ...
col_to_delete_contactos=list()

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_seniority__c: Variable numerica <br>
</div>

In [13]:
# Vamos a realizar analisis por cada variable
var = "msf_seniority__c"

In [14]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_seniority__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_seniority__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [15]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

      # Tot     % Tot
0.0   72843  7.327798
8.0   69373  6.978726
7.0   68120  6.852677
6.0   65874  6.626736
9.0   64102  6.448478
5.0   47211  4.749292
10.0  43910  4.417221
12.0  43340  4.359880
4.0   39932  4.017045
13.0  38984  3.921679
11.0  37505  3.772896
14.0  36133  3.634877
18.0  34795  3.500278
2.0   31380  3.156738
17.0  31370  3.155732
1.0   30543  3.072539
16.0  30132  3.031193
15.0  28826  2.899813
3.0   27544  2.770848
19.0  26257  2.641379
29.0  19570  1.968686
20.0  18961  1.907422
23.0  13852  1.393472
22.0  10824  1.088863
21.0   9968  1.002752
24.0   9882  0.994101
28.0   9759  0.981728
25.0   9327  0.938270
27.0   6960  0.700156
31.0   6128  0.616459
26.0   4656  0.468380
30.0   4424  0.445042
32.0    881  0.088626
35.0    216  0.021729
34.0    215  0.021628
33.0    171  0.017202
36.0     84  0.008450
37.0     12  0.001207


<div class="alert alert-block alert-success">
<b>msf_seniority__c: Número de años desde la fecha de su primera aportación económica hasta día de hoy.</b> <br>
Se puede observar que está bastante distribuido, siendo en 0 donde se acumula la mayor parte de la población. Se analizará posteriormente si la categorización de la variable en grupos pueda dar buenos resultados. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__best_gift_year__c: Variable numerica <br>
</div>

In [16]:
# Vamos a realizar analisis por cada variable
var = "npo02__best_gift_year__c"

In [17]:
# Analizamos nulos
count_nulos(df_contactos,var,col_to_delete_contactos)

El nº de nulos para la variable npo02__best_gift_year__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npo02__best_gift_year__c es 709207. Lo que supone un 39.32569192184401%
['npo02__best_gift_year__c']


In [18]:
# Analizamos posibles valores de la variable
freq_variables(df_contactos,var)

       # Tot      % Tot
      709207  39.325692
2018  303667  16.838405
2022  185032  10.260067
2021   93074   5.160975
2020   90828   5.036434
2019   77054   4.272662
2023   55899   3.099612
2010   29210   1.619701
1994   28224   1.565027
2017   21245   1.178040
2005   15932   0.883433
2014   14681   0.814065
2011   14643   0.811958
2004   13160   0.729725
2000   12659   0.701944
2015   11996   0.665181
2001   11403   0.632299
1998   11363   0.630081
2013   10940   0.606626
2016    9948   0.551619
2003    9537   0.528829
2008    8465   0.469386
1999    8142   0.451476
2009    7599   0.421366
1996    6869   0.380888
2012    6795   0.376784
2006    6723   0.372792
1992    6238   0.345899
2007    5562   0.308414
2002    4753   0.263555
1997    4491   0.249027
1995    4064   0.225350
1993    2470   0.136962
1991     624   0.034601
1989     435   0.024121
1990     212   0.011755
1988     187   0.010369
1987      88   0.004880


<div class="alert alert-block alert-success">
<b>npo02__best_gift_year__c: Año fiscal en que se ha realizado mayor importe total.</b> <br>
Se puede observar que hay casi un 40% de los registros a vacio. Se analizará posteriormente por si son clientes sin ninguna aportacion y poder analizar si es vacio con sentido o no.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_birthyear__c: Variable numerica <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_birthyear__c"

In [20]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [21]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_birthyear__c: .</b> <br>
Se puede observar que hay más de un 20% de los registros a vacio. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_entrycampaign__c: Variable string <br>
</div>

In [22]:
# Vamos a realizar analisis por cada variable
var = "msf_entrycampaign__c"

In [23]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_entrycampaign__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_entrycampaign__c es 74. Lo que supone un 0.007444188704147821%


In [24]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                    # Tot     % Tot
7013Y000001mr4CQAQ  37526  3.775008
7013Y000001mrCzQAI  36910  3.713041
7013Y000001mr2DQAQ  30523  3.070527
7013Y000001mr2cQAA  25776  2.592992
7013Y000001mrBSQAY  24519  2.466541
7013Y000001mr4AQAQ  22386  2.251968
7013Y000001mr25QAA  22272  2.240500
7013Y000001mr1MQAQ  22009  2.214043
7013Y000001mrBoQAI  21890  2.202071
7013Y000001mr1mQAA  17326  1.742946
7013Y000001mr2CQAQ  13674  1.375565
7013Y000001mr2TQAQ  12482  1.255654
7013Y000001mrC7QAI  11594  1.166323
7013Y000001mr1sQAA  11558  1.162702
7013Y000001mqq8QAA  10895  1.096006
7013Y000001mqvKQAQ   9934  0.999332
7013Y000001mr2RQAQ   9909  0.996817
7013Y000001mr3eQAA   9409  0.946519
7013Y000001mrBhQAI   9363  0.941891
7013Y000001mr2KQAQ   9339  0.939477
7013Y000001mr2tQAA   9135  0.918955
7013Y000001mr1kQAA   8655  0.870668
7013Y000001mr2sQAA   8565  0.861615
7013Y000001mr1jQAA   8544  0.859502
7013Y000001mr2eQAA   8466  0.851655
7013Y000001mr1uQAA   8216  0.826506
7013Y000001mrEJQAY   7962  0

<div class="alert alert-block alert-success">
<b>msf_birthyear__c: .</b> <br>
Se puede observar que hay más de un 20% de los registros a vacio. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> LeadSource: Variable categorica <br>
</div>

In [25]:
# Vamos a realizar analisis por cada variable
var = "leadsource"

In [26]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable leadsource es 0. Lo que supone un 0.0%
El nº de vacios para la variable leadsource es 20. Lo que supone un 0.0020119428930129245%


In [27]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                                 # Tot      % Tot
Persona a persona               346349  34.841720
Otro                            229312  23.068132
Telemarketing                   153973  15.489244
Personal con tablet              70560   7.098135
Cupón                            65399   6.578953
Web MSF                          64678   6.506422
Teléfono campaña                 34878   3.508627
Web terceros                     14767   1.485518
Web campaña                       4795   0.482363
Teléfono web                      4304   0.432970
Eventos                           1969   0.198076
Teléfono SAS                      1524   0.153310
Email a SAS                        921   0.092650
Email a Empresas                   138   0.013882
Plataforma iniciativas             127   0.012776
Email a Bodas                      124   0.012474
Correo postal sin cupón             92   0.009255
Entidad financiera                  87   0.008752
Teléfono Officers                   23   0.002314


<div class="alert alert-block alert-success">
<b>leadsource: Canal principal.</b> <br>
Se puede observar que casi no hay vacios. La mayor parte es Persona a Persona. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_firstcampaigncolaborationchannel__c: Variable categorica <br>
</div>

In [28]:
# Vamos a realizar analisis por cada variable
var = "msf_firstcampaigncolaborationchannel__c"

In [29]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_firstcampaigncolaborationchannel__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_firstcampaigncolaborationchannel__c es 139277. Lo que supone un 14.010868515508058%


In [30]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

                                 # Tot      % Tot
Persona a persona               310662  31.251710
Telemarketing                   162367  16.333657
                                139277  14.010869
Otro                            113431  11.410835
Cupón                            67920   6.832558
Web MSF                          67860   6.826522
Personal con tablet              65618   6.600983
Teléfono campaña                 36839   3.705898
Web terceros                     12878   1.295490
Teléfono web                      4501   0.452788
Web campaña                       4339   0.436491
Teléfono SAS                      2498   0.251292
Email a SAS                       1437   0.144558
Eventos                           1435   0.144357
Plataforma iniciativas            1006   0.101201
web campaña                        624   0.062773
Entidad financiera                 583   0.058648
cupón                              224   0.022534
Web MSF Mi perfil                  141   0.014184


<div class="alert alert-block alert-success">
<b>msf_firstcampaigncolaborationchannel__c: Canal por el que realizó la primera donación.</b> <br>
Se puede observar que hay un 15% de vacios. Como estamos analizando la poblacion de donaciones recurrentes, el que no haya realizado ninguna donación, no tiene sentido, por lo que son vacios. La información se parece a la del campo LeadSource, y tiene más vacios, por lo que nos quedaremos con la anterior para el modelo. 
</div>

In [31]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_contactos.append("msf_firstcampaigncolaborationchannel__c")
col_to_delete_contactos

['npo02__best_gift_year__c', 'msf_birthyear__c', 'msf_firstcampaigncolaborationchannel__c']


<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__AverageAmount__c: Variable numerica <br>
</div>

In [35]:
# Vamos a realizar analisis por cada variable
var = "npo02__averageamount__c"

In [36]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable npo02__averageamount__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npo02__averageamount__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [37]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

      # Tot  % Tot
0.0  994064  100.0


<div class="alert alert-block alert-success">
<b>npo02__averageamount__c: Media del total de aportaciones.</b> <br>
Se puede observar que no hay vacios pero está informado a 0 para todos los casos. 
</div>

In [38]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_contactos.append("npo02__averageamount__c")
col_to_delete_contactos

['npo02__best_gift_year__c', 'msf_birthyear__c', 'msf_firstcampaigncolaborationchannel__c', 'npo02__averageamount__c']


<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
        -msf_isactivedonor__c: Variable categorica <br>
</div>

In [39]:
# Vamos a realizar analisis por cada variable
var = "msf_isactivedonor__c"

In [40]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_isactivedonor__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_isactivedonor__c es 0. Lo que supone un 0.0%


In [41]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

            # Tot      % Tot
Nunca      727574  73.191867
Exdonante  212374  21.364218
Donante     54116   5.443915


<div class="alert alert-block alert-success">
<b>msf_isactivedonor__c: XXXXXXXXX.</b> <br>
Se puede observar que no hay vacios, aunque que el campo se informe como nunca en la mayor parte de los casos, para los clientes comunes con la tabla de recurring donation no tiene mucho sentido, habrá que analizar en conjunto con otras variables de esa tabla. 
</div>

In [45]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_contactos.append("msf_isactivedonor__c")
col_to_delete_contactos

['npo02__best_gift_year__c', 'msf_birthyear__c', 'msf_firstcampaigncolaborationchannel__c', 'npo02__averageamount__c', 'msf_isactivedonor__c']


<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_isactiverecurringdonor__c: Variable categorica <br>
</div>

In [42]:
# Vamos a realizar analisis por cada variable
var = "msf_isactiverecurringdonor__c"

In [43]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_isactiverecurringdonor__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_isactiverecurringdonor__c es 0. Lo que supone un 0.0%


In [44]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

        # Tot      % Tot
Baja   511080  51.413189
Socio  482224  48.510357
Nunca     760   0.076454


<div class="alert alert-block alert-success">
<b>msf_isactiverecurringdonor__c: indicador de socio recurrente.</b> <br>
Se puede observar que no hay vacios, pero no añade más información que el campo de activo de la tabla de recurring donation. Por lo que esta variable y la anterior se eliminarán de esta tabla. 
</div>

In [46]:
# Se va a añadir esta variable a la lista de columnas a borrar
col_to_delete_contactos.append("msf_isactiverecurringdonor__c")
col_to_delete_contactos

['npo02__best_gift_year__c', 'msf_birthyear__c', 'msf_firstcampaigncolaborationchannel__c', 'npo02__averageamount__c', 'msf_isactivedonor__c', 'msf_isactiverecurringdonor__c']


<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp__deceased__c: Variable categorica <br>
</div>

In [47]:
# Vamos a realizar analisis por cada variable
var = "npsp__deceased__c"

In [48]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable npsp__deceased__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable npsp__deceased__c es 0. Lo que supone un 0.0%
/home/spark/.local/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [49]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

        # Tot      % Tot
False  971783  97.758595
True    22281   2.241405


<div class="alert alert-block alert-success">
<b>npsp__deceased__c: Indicador de fallecido</b> <br>
Se puede observar que no hay vacios, solo el 2% han fallecido. Esto quiere decir que se podrán usar en el modelo para prdecir, pero no para aplicar.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_begindatemsf__c: Variable categorica <br>
</div>

In [50]:
# Vamos a realizar analisis por cada variable
var = "msf_begindatemsf__c"

In [51]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_begindatemsf__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_begindatemsf__c es 0. Lo que supone un 0.0%


In [52]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

            # Tot     % Tot
2004-01-01   4833  0.486186
2000-02-01   4462  0.448864
2000-01-01   3903  0.392631
1994-10-01   3486  0.350682
1995-02-01   3386  0.340622
1998-03-01   2617  0.263263
1994-02-01   2568  0.258333
1999-01-01   2523  0.253807
2004-02-01   2477  0.249179
2013-03-28   2092  0.210449
1999-02-01   2085  0.209745
1996-02-01   2036  0.204816
1997-02-01   1983  0.199484
2000-03-01   1856  0.186708
1994-07-01   1712  0.172222
2015-12-22   1682  0.169204
2001-12-31   1551  0.156026
1998-02-01   1506  0.151499
1992-11-01   1476  0.148481
2014-02-27   1399  0.140735
1994-01-01   1393  0.140132
2014-10-16   1387  0.139528
2005-02-02   1359  0.136712
2013-08-28   1357  0.136510
2001-02-01   1356  0.136410
1994-09-01   1338  0.134599
2015-02-25   1330  0.133794
1995-04-01   1307  0.131480
2016-07-28   1301  0.130877
1995-03-01   1280  0.128764
1998-01-01   1274  0.128161
2014-05-23   1264  0.127155
2001-01-01   1263  0.127054
2005-02-03   1243  0.125042
2015-09-30   1198  0

<div class="alert alert-block alert-success">
<b>msf_begindatemsf__c: Fecha de entrada en MSF.</b> <br>
Se puede observar que no hay vacios, se podrá tranformar en "tiempo en MSF" teniendo en cuenta la fecha de baja y "tiempo desde entrada hasta donación" quizá pueda ser util en el modelo.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_fechacambiolevelrelacion__c: Variable categorica <br>
</div>

In [53]:
# Vamos a realizar analisis por cada variable
var = "msf_fechacambiolevelrelacion__c"

In [54]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_fechacambiolevelrelacion__c es 4. Lo que supone un 0.00040238857860258495%
El nº de vacios para la variable msf_fechacambiolevelrelacion__c es 0. Lo que supone un 0.0%


In [55]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

             # Tot      % Tot
2020-03-28  852024  85.711526
2020-07-20    6964   0.700561
2020-09-22    4511   0.453796
2020-09-19    3664   0.368589
2022-01-02    2462   0.247671
2020-09-21    1639   0.164879
2023-01-03    1481   0.148985
2020-09-20    1466   0.147476
2021-01-04    1023   0.102911
2023-01-02     777   0.078164
2022-01-15     707   0.071122
2022-05-06     536   0.053920
2022-03-22     529   0.053216
2020-12-04     522   0.052512
2022-06-04     491   0.049393
2021-06-18     472   0.047482
2022-12-03     461   0.046375
2022-05-11     445   0.044766
2021-03-04     439   0.044162
2022-12-23     434   0.043659
2020-10-03     431   0.043358
2020-09-23     425   0.042754
2022-10-21     418   0.042050
2023-02-10     403   0.040541
2021-07-08     398   0.040038
2021-02-05     395   0.039736
2022-09-23     387   0.038931
2021-08-07     380   0.038227
2022-02-05     379   0.038126
2022-11-24     376   0.037825
2021-06-25     374   0.037623
2021-11-18     373   0.037523
2020-09-25

<div class="alert alert-block alert-success">
<b>msf_fechacambiolevelrelacion__c: Fecha de cambio de nivel de relación.</b> <br>
Se puede observar que aunque practicamente no hay vacios ni nulos, el 85% de la muestra tiene fecha
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_datefirstdonation__c: Variable fecha <br>
</div>

In [3]:
# Vamos a realizar analisis por cada variable
var = "msf_datefirstdonation__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_datefirstdonation__c: Fecha de la primera donacion.</b> <br>
Se puede observar que hay más de un 20% de los registros a vacio. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_datefirstrecurringdonorquota__c: Variable fecha <br>
</div>

In [2]:
# Vamos a realizar analisis por cada variable
var = "msf_datefirstrecurringdonorquota__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_datefirstrecurringdonorquota__c: Fecha de la primera donacion recurrente.</b> <br>
Se puede observar que apenas hay vacios. Se puede usar para tener variables como tiempo desde primera donacion recurrente o tiempo hasta modificaicon de cuota. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_datelastrecurringdonorquota__c: Variable fecha <br>
</div>

In [1]:
# Vamos a realizar analisis por cada variable
var = "msf_datelastrecurringdonorquota__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_datelastrecurringdonorquota__c: Fecha de la ultima donacion recurrente.</b> <br>
Se puede observar que .... vacios. En la tabla de donaciones recurrentes tambien hay una fecha de ultima donacion recurrente, por lo que esta no aporta informacion adicional.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_datelastdonation__c: Variable fecha <br>
</div>

In [4]:
# Vamos a realizar analisis por cada variable
var = "msf_datelastdonation__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_datelastdonation__c: Fecha de la ultima donacion.</b> <br>
Se puede observar que .... vacios. Al igual que no se tiene en cuenta la de la primera, tampoco la de la ultima.
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp__largest_soft_credit_date__c: Variable fecha <br>
</div>

In [5]:
# Vamos a realizar analisis por cada variable
var = "npsp__largest_soft_credit_date__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npsp__largest_soft_credit_date__c: Fecha de la aportacion indirecta más importante.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp__first_soft_credit_date__c: Variable fecha <br>
</div>

In [6]:
# Vamos a realizar analisis por cada variable
var = "npsp__first_soft_credit_date__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npsp__first_soft_credit_date__c: Fecha de la primera aportación indirecta.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_entrydatecurrentrecurringdonor__c: Variable fecha <br>
</div>

In [7]:
# Vamos a realizar analisis por cada variable
var = "msf_entrydatecurrentrecurringdonor__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_entrydatecurrentrecurringdonor__c: Fecha de la ultima entrada de socio.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp__last_soft_credit_date__c: Variable fecha <br>
</div>

In [8]:
# Vamos a realizar analisis por cada variable
var = "npsp__last_soft_credit_date__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npsp__last_soft_credit_date__c: Fecha de la ultima aportación indirecta.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_firstentrydaterecurringdonor__c: Variable fecha <br>
</div>

In [9]:
# Vamos a realizar analisis por cada variable
var = "msf_firstentrydaterecurringdonor__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_firstentrydaterecurringdonor__c: Fecha de la primera entrada de socio.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__firstclosedate__c: Variable fecha <br>
</div>

In [10]:
# Vamos a realizar analisis por cada variable
var = "npo02__firstclosedate__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__firstclosedate__c: Fecha de la primera donación de cualquier tipo.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_lastrecurringdonationdate__c: Variable fecha <br>
</div>

In [11]:
# Vamos a realizar analisis por cada variable
var = "msf_lastrecurringdonationdate__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_lastrecurringdonationdate__c: Fecha de la ultima baja de socio.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__lastclosedate__c: Variable fecha <br>
</div>

In [12]:
# Vamos a realizar analisis por cada variable
var = "npo02__lastclosedate__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__lastclosedate__c: Fecha de la ultima donación de cualquier tipo.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> gender__c: Variable categorica <br>
</div>

In [12]:
# Vamos a realizar analisis por cada variable
var = "gender__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>gender__c: Genero.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_languagepreferer__c: Variable categorica <br>
</div>

In [13]:
# Vamos a realizar analisis por cada variable
var = "msf_languagepreferer__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_languagepreferer__c: lenguaje de comunicacion.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__largestamount__c: Variable numerica <br>
</div>

In [13]:
# Vamos a realizar analisis por cada variable
var = "npo02__largestamount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__largestamount__c: importe de la donacion más grande.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__smallestamount__c: Variable numerica <br>
</div>

In [13]:
# Vamos a realizar analisis por cada variable
var = "npo02__smallestamount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__smallestamount__c: importe de la donacion más pequeña.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp__first_soft_credit_amount__c: Variable numerica <br>
</div>

In [13]:
# Vamos a realizar analisis por cada variable
var = "npsp__first_soft_credit_amount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npsp__first_soft_credit_amount__c: importe de la primera aportacion indirecta.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__lastoppamount__c: Variable numerica <br>
</div>

In [13]:
# Vamos a realizar analisis por cada variable
var = "npo02__lastoppamount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__lastoppamount__c: importe de la ultima aportacion.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp__last_soft_credit_amount__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "npsp__last_soft_credit_amount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npsp__last_soft_credit_amount__c: importe de la ultima aportacion indirecta.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_annualizedquotachange__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_annualizedquotachange__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_annualizedquotachange__c: incremento de cuota anualizado que se le pediria.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_relationshiplevel__c: Variable categorica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_relationshiplevel__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_relationshiplevel__c: tipo de relacion que se desea con el contacto.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_ltvcont__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_ltvcont__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_ltvcont__c: valor de todas las aportaciones.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> mailingstate: Variable categorica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "mailingstate"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>mailingstate: provincia.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npsp__largest_soft_credit_amount__c: Variable numerica <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "npsp__largest_soft_credit_amount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npsp__largest_soft_credit_amount__c: mayor importe de operaciones indirectas.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__soft_credit_last_year__c: Variable numerica <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "npo02__soft_credit_last_year__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__soft_credit_last_year__c: operaciones indirectas el año pasado.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__soft_credit_this_year__c: Variable numerica <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "npo02__soft_credit_this_year__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__soft_credit_this_year__c: operaciones indirectas este año.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__soft_credit_two_years_ago__c: Variable numerica <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "npo02__soft_credit_two_years_ago__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__soft_credit_two_years_ago__c: operaciones indirectas hace 2 años.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_nocaptacionfondoscp__c: Variable booleana <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "msf_nocaptacionfondoscp__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_nocaptacionfondoscp__c: permiso de comuncacion por correo postal.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_nocaptacionfondosemail__c: Variable booleana <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "msf_nocaptacionfondosemail__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_nocaptacionfondosemail__c: permiso de comuncacion por email.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_nocaptacionfondosmi__c: Variable booleana <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "msf_nocaptacionfondosmi__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_nocaptacionfondosmi__c: permiso de comuncacion por mi.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_nocaptacionfondossms__c: Variable booleana <br>
</div>

In [20]:
# Vamos a realizar analisis por cada variable
var = "msf_nocaptacionfondossms__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_nocaptacionfondossms__c: permiso de comuncacion por sms.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_firstcampaignentryrecurringdonor__c: Variable categorica <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_firstcampaignentryrecurringdonor__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_firstcampaignentryrecurringdonor__c: primera campaña de colaboracion como socio recurrente.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_firstcampaingcolaboration__c: Variable categorica <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_firstcampaingcolaboration__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_firstcampaingcolaboration__c: primera campaña de colaboracion economica.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_firstannualizedquota__c: Variable numerica <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_firstannualizedquota__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_firstannualizedquota__c: importe anualizado del primer compromiso como socio.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_program__c: Variable categorica <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_program__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_program__c: programa al que pertenece.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_programaherencias__c: Variable booleana <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_programaherencias__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_programaherencias__c: indicador de algun tipo de relacion con el programa de herencias.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_programais__c: Variable booleana <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_programais__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_programais__c: indicador de promotor en iniciativa solidaria.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_pressurecomplaint__c: Variable booleana <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_pressurecomplaint__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_pressurecomplaint__c: queja por presión telemarketing.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_recencydonorcont__c: Variable numerica <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_recencydonorcont__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_recencydonorcont__c: numero de dias desde el ultimo donativo.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_recencyrecurringdonorcont__c: Variable numerica <br>
</div>

In [19]:
# Vamos a realizar analisis por cada variable
var = "msf_recencyrecurringdonorcont__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_recencyrecurringdonorcont__c: numero de dias desde la ultima aportacion de socio recurrente.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_recencytotalcont__c: Variable numerica <br>
</div>

In [18]:
# Vamos a realizar analisis por cada variable
var = "msf_recencytotalcont__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_recencytotalcont__c: numero de dias desde la ultima aportacion.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_scoringrfvdonor__c: Variable numerica <br>
</div>

In [17]:
# Vamos a realizar analisis por cada variable
var = "msf_scoringrfvdonor__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_scoringrfvdonor__c: scoring donante.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_scoringrfvrecurringdonor__c: Variable numerica <br>
</div>

In [17]:
# Vamos a realizar analisis por cada variable
var = "msf_scoringrfvrecurringdonor__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_scoringrfvrecurringdonor__c: scoring donante recurrente.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_scoringrvtotal__c: Variable numerica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "msf_scoringrvtotal__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_scoringrvtotal__c: scoring total.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_mailingsegment__c: Variable categorica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "msf_mailingsegment__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_mailingsegment__c: segmento colaborador.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_membertype__c: Variable categorica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "msf_membertype__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_membertype__c: tipo de miembro.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__totaloppamount__c: Variable numerica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "npo02__totaloppamount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__totaloppamount__c: total donado.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__oppamountthisyear__c: Variable numerica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "npo02__OppAmountThisYear__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__OppAmountThisYear__c: importe total de aportaciones al año que realizó este año.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__oppamount2yearsago__c: Variable numerica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "npo02__oppamount2yearsago__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__oppamount2yearsago__c: importe total de aportaciones al año que realizó hace 2 años.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__oppamountlastyear__c: Variable numerica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "npo02__oppamountlastyear__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__oppamountlastyear__c: importe total de aportaciones al año que realizó el año pasado.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> npo02__best_gift_year_total__c: Variable numerica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "npo02__best_gift_year_total__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>npo02__best_gift_year_total__c: importe total de aportaciones al año que más ha aportado.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_totalfiscaloppamount__c: Variable numerica <br>
</div>

In [15]:
# Vamos a realizar analisis por cada variable
var = "msf_totalfiscaloppamount__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_totalfiscaloppamount__c: importe total de aportaciones fiscal cobradas.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_lastannualizedquota__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_lastannualizedquota__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_lastannualizedquota__c: importe anualizado de la ultima cuota de socio.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_valuetotalcont__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_valuetotalcont__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_valuetotalcont__c: valor colaborador.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_valuedonorcont__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_valuedonorcont__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_valuedonorcont__c: suma de las donaciones de los ultimoss 365 dias.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_lastyeardonorvalue__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_lastyeardonorvalue__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_lastyeardonorvalue__c: suma de las aportaciones de los ultimos 365 dias.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_maximumdonorvalue__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_maximumdonorvalue__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_maximumdonorvalue__c: importe más elevado de todos los donativos.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_averagedonorvalue__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_averagedonorvalue__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_averagedonorvalue__c: importe medio de todos los donativos.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_lifetime__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_lifetime__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_lifetime__c: numero de años enteros desde primera aportacion a la ultima.</b> <br>
Se puede observar que .... vacios. 
</div>

<div class="alert alert-block alert-info">
<b>Analsis de distribución por variables</b> <br>
         -> msf_commitment__c: Variable numerica <br>
</div>

In [14]:
# Vamos a realizar analisis por cada variable
var = "msf_commitment__c"

In [118]:
# Analizamos nulos
count_nulos(df,var,col_to_delete_contactos)

El nº de nulos para la variable msf_birthyear__c es 0. Lo que supone un 0.0%
El nº de vacios para la variable msf_birthyear__c es 216719. Lo que supone un 21.8013125915434%
['npo02__best_gift_year__c', 'msf_birthyear__c']


In [119]:
# Analizamos posibles valores de la variable
freq_variables(df,var)

       # Tot      % Tot
      216719  21.801313
1964   17594   1.769906
1963   17337   1.744053
1965   17330   1.743349
1968   17142   1.724436
1959   17014   1.711560
1966   16945   1.704619
1958   16925   1.702607
1962   16912   1.701299
1973   16892   1.699287
1975   16872   1.697275
1974   16840   1.694056
1961   16738   1.683795
1960   16698   1.679771
1972   16694   1.679369
1957   16680   1.677960
1967   16658   1.675747
1976   16577   1.667599
1969   16530   1.662871
1971   16525   1.662368
1970   16374   1.647178
1977   16072   1.616797
1978   15831   1.592553
1956   15179   1.526964
1979   15167   1.525757
1980   14467   1.455339
1955   14076   1.416005
1981   13669   1.375062
1954   13015   1.309272
1982   12607   1.268228
1953   12371   1.244487
1983   11938   1.200929
1952   11928   1.199923
1984   11031   1.109687
1951   10886   1.095101
1950   10448   1.051039
1985   10209   1.026996
1949   10096   1.015629
1948    9717   0.977502
1986    9190   0.924488
1987    8599   0

<div class="alert alert-block alert-success">
<b>msf_commitment__c: suma de iteraciones.</b> <br>
Se puede observar que .... vacios. 
</div>

# Se va a unir la información de la tabla contactos a la tabla de recurring donation

In [78]:
df_recurring_plus=pd.merge(df_re_donation,df_contactos,how='left', left_on='npe03__contact__c', right_on='id')

In [79]:
df_recurring_plus.count()

id_x                                          1198207
isdeleted                                     1198207
msf_annualizedquota__c                        1198207
msf_cancelationdate__c                         715201
msf_cancelationreason__c                      1198207
msf_currentcampaign__c                        1198207
msf_currentleadsource1__c                     1198207
msf_currentquotamodification__c               1198207
msf_leadsource1__c                            1198207
msf_memberid__c                               1198207
npe03__amount__c                              1198207
npe03__contact__c                             1198207
npe03__date_established__c                    1198207
npe03__installment_period__c                  1198207
npe03__last_payment_date__c                   1122597
npe03__next_payment_date__c                    483302
npe03__open_ended_status__c                   1198207
npe03__paid_amount__c                         1196308
npe03__recurring_donation_ca